## Import libraries
- request
- bs4/BeautifulSoup
- pandas

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [2]:
resp = requests.get(url)

## Get all squad links
1. Find the table with `table` element with `class= "stats_table"`
2. Find all links of the table rows (`get 'href'`)
3. Check link is start with `/en/squads/` or then add to list


In [3]:
soup = BeautifulSoup(resp.text, 'html.parser')              
standing_table = soup.find_all('table', class_="stats_table")[0]
links = standing_table.find_all('a')
links = [link.get('href') for link in links]
links = [link for link in links if link.startswith('/en/squads/')]
links 

['/en/squads/b8fd03ef/Manchester-City-Stats',
 '/en/squads/18bb7c10/Arsenal-Stats',
 '/en/squads/822bd0ba/Liverpool-Stats',
 '/en/squads/8602292d/Aston-Villa-Stats',
 '/en/squads/361ca564/Tottenham-Hotspur-Stats',
 '/en/squads/cff3d9bb/Chelsea-Stats',
 '/en/squads/b2b47a98/Newcastle-United-Stats',
 '/en/squads/19538871/Manchester-United-Stats',
 '/en/squads/7c21e445/West-Ham-United-Stats',
 '/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 '/en/squads/4ba7cbea/Bournemouth-Stats',
 '/en/squads/47c64c55/Crystal-Palace-Stats',
 '/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 '/en/squads/fd962109/Fulham-Stats',
 '/en/squads/d3fd31cc/Everton-Stats',
 '/en/squads/cd051869/Brentford-Stats',
 '/en/squads/e4a775cb/Nottingham-Forest-Stats',
 '/en/squads/e297cd13/Luton-Town-Stats',
 '/en/squads/943e8050/Burnley-Stats',
 '/en/squads/1df6b87e/Sheffield-United-Stats']

In [4]:
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/s

## Scraping squad games

In [5]:
RESULT = []
for team_url in team_urls:
    team_name = team_url.split('/')[-1].replace("-Stats",'').replace('-',' ')
    data = requests.get(team_url)
    
    matches = pd.read_html(data.text,match="Scores & Fixtures")[0]
    
    soup = BeautifulSoup(data.text, 'html.parser')
    links = [link.get('href') for link in soup.find_all('a')]
    links = [link for link in links if link and 'all_comps/shooting/' in link]
    
    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data.text,match='Shooting')[0]
    shooting.columns = shooting.columns.droplevel()
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    except ValueError:
        continue
    team_data = team_data[team_data['Comp'] == 'Premier League']
    team_data['Season'] = 2024 
    team_data['Team'] = team_name
    RESULT.append(team_data)      

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7964\2057238521.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text,match="Scores & Fixtures")[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7964\2057238521.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text,match='Shooting')[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7964\2057238521.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text,match="Scores & Fixtures")[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7964\2057238521.py:13: FutureWarning: Passing literal html to '

## Write it into csv

In [6]:
import datetime
RESULT_DF = pd.concat(RESULT)
RESULT_DF.columns = [c.lower() for c in RESULT_DF.columns]
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
RESULT_DF.to_csv(f'matches{current_date}.csv', index=False)